# SageMaker TensorFlow-Kerra Pipeline

In [1]:
# !pip install -U sagemaker

In [2]:
import sagemaker
import boto3
import os
import json
import numpy as np
import time
import tensorflow as tf
from sagemaker.tensorflow import TensorFlow
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import CacheConfig, ProcessingStep, TrainingStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.parameters import ParameterFloat, ParameterString
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
from sagemaker.workflow.properties import PropertyFile
from sagemaker.model import Model
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model_metrics import ModelMetrics, MetricsSource

from sagemaker.tensorflow import TensorFlowProcessor

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


2025-02-19 23:09:43.917626: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Define variables

In [3]:
# Initialize session
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.session.Session().region_name
pipeline_session = PipelineSession()
s3_client = boto3.client("s3")
# s3_bucket = sagemaker_session.default_bucket()
bucket = "aamlops2024"
prefix = "tensorflow-sagemaker-pipeline"
s3_base_uri = f"s3://{bucket}/{prefix}"
bucket, prefix, s3_base_uri, region

('aamlops2024',
 'tensorflow-sagemaker-pipeline',
 's3://aamlops2024/tensorflow-sagemaker-pipeline',
 'us-east-1')

In [4]:
# Parameters
processing_instance_type = ParameterString(name="ProcessingInstanceType", default_value="ml.m5.large")
training_instance_type = ParameterString(name="TrainingInstanceType", default_value="ml.m5.large")
model_approval_status = ParameterString(name="ModelApprovalStatus", default_value="PendingManualApproval")
accuracy_threshold = ParameterFloat(name="AccuracyThreshold", default_value=0.75)
model_package_group_name = "TensorFlowModel"

In [5]:
# S3 paths
data_location = f"s3://{bucket}/{prefix}/processing/"
model_output_path = f"s3://{bucket}/{prefix}/artifacts/"

In [6]:
tensorflow_image_uri_cpu = "763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12-cpu-py310"
tensorflow_image_uri_gpu = "763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12-gpu-py310"

## 0 Data Preparation

In [7]:
input_data_uri = f"s3://{bucket}/{prefix}/data/rawdata1.csv"
input_data_uri

's3://aamlops2024/tensorflow-sagemaker-pipeline/data/rawdata1.csv'

In [8]:
# Define cache configuration
cache_config = CacheConfig(
    enable_caching=True,             # Enable caching
    expire_after="P30D"              # Cache expiry in ISO 8601 duration format (e.g., P30D = 30 days)
)

## 1/ Model Preprocessing

In [9]:
# processor = ScriptProcessor(
#     image_uri=tensorflow_image_uri_cpu,
#     role=role,
#     sagemaker_session=pipeline_session,
#     instance_count=1,
#     instance_type="ml.c5.xlarge"
# )

In [10]:
tensorFlow_Processor = TensorFlowProcessor(
    framework_version='2.3',
    role=role,
    sagemaker_session=pipeline_session,
    instance_type='ml.m5.xlarge',
    instance_count=1,
    base_job_name='frameworkprocessor-TF',
    py_version='py37'
)

[02/19/25 23:09:47] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=308026;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=419455;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

In [11]:
procesor_args = tensorFlow_Processor.run(
    inputs=[
        ProcessingInput(source=input_data_uri, destination="/opt/ml/processing/input")
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=f"s3://{bucket}/{prefix}/processing"),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=f"s3://{bucket}/{prefix}/processing"),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=f"s3://{bucket}/{prefix}/processing"),
    ],
    code="./code/preprocess.py"
)

/opt/conda/lib/python3.11/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [12]:
step_process = ProcessingStep (name="PreprocessData", step_args=procesor_args, cache_config=cache_config)

## 2️⃣ Step_Train - Train TensorFlow Model

In [13]:
tensorflow_estimator = TensorFlow(
    entry_point="./code/train.py",
    role=role,
    sagemaker_session=pipeline_session,
    instance_count=1,
    instance_type=training_instance_type,
    framework_version="2.12",
    py_version="py310",
    script_mode=True,
    hyperparameters={"epochs": 10},
    output_path=model_output_path,
)

In [14]:
train_args = tensorflow_estimator.fit(
    inputs={
        "train": TrainingInput(s3_data=step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri, content_type="text/csv"),
        "validation": TrainingInput(s3_data=step_process.properties.ProcessingOutputConfig.Outputs["validation"].S3Output.S3Uri, content_type="text/csv")
    },
)

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=553206;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=201999;file:///opt/conda/lib/python3.11/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

In [15]:
step_train = TrainingStep (name="TrainTensorFlowModel", step_args=train_args, cache_config=cache_config)

## 3️⃣ Step_Evaluation - Evaluate Model

In [16]:
tensorFlow_eval = TensorFlowProcessor(
    framework_version='2.12',
    role=role,
    sagemaker_session=pipeline_session,
    instance_type='ml.m5.large',
    instance_count=1,
    base_job_name='frameworkprocessor-TF',
    py_version='py310'
)

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=895228;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=595435;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

In [17]:
eval_args = tensorFlow_eval.run(
    inputs=[
        ProcessingInput(source=step_train.properties.ModelArtifacts.S3ModelArtifacts, destination="/opt/ml/processing/model"),
        ProcessingInput(source=step_process.properties.ProcessingOutputConfig.Outputs["test"].S3Output.S3Uri, destination="/opt/ml/processing/test"),
    ],
    outputs=[ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation", destination=f"s3://{bucket}/{prefix}/evaluation")],
    code="./code/evaluate.py"  # your custom evaluation script
)

evaluation_report = PropertyFile(
    name="EvaluationReport", output_name="evaluation", path="evaluation.json",
)

In [18]:
step_eval = ProcessingStep(name="EvaluateModel", step_args=eval_args, property_files=[evaluation_report], cache_config=cache_config )

## 4️⃣ Step Register

In [19]:
model_metrics = ModelMetrics(
  model_statistics=MetricsSource(
    s3_uri="{}/evaluation.json".format(
      step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"]["S3Uri"]
    ),
    content_type="application/json"
  )
)

[02/19/25 23:09:48] INFO     Uploaded None to                                                    ]8;id=814783;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=616416;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#1961\1961]8;;\
                             s3://sagemaker-us-east-1-047922237497/frameworkprocessor-TF-2025-02                   
                             -19-23-09-47-966/source/sourcedir.tar.gz                                              

                    INFO     runproc.sh uploaded to                                              ]8;id=389723;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=67312;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#2055\2055]8;;\
                             s3://sagemaker-us-east-1-047922237497/frameworkprocessor-TF-2025-02                   
                             -19-23-09-47-966/source/runproc.sh                                                    

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=582111;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=666558;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

In [20]:
step_register = RegisterModel(
    name="TensorFlow",
    estimator=tensorflow_estimator,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    content_types=["application/x-tf-savedmodel"],
    response_types=["application/json"],
    inference_instances=["ml.t2.medium", "ml.t2.medium"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
    depends_on=[step_eval]
)

## 5️⃣ Step Condition - Check Accuracy

In [21]:
step_fail = FailStep(
    name="TrainingFailed",
    error_message="Model accuracy did not meet the required threshold.",
)

In [22]:
cond_lte = ConditionGreaterThanOrEqualTo(  # You can change the condition here
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="binary_classification_metrics.accuracy.value"  # This should follow the structure of your report_dict defined in the evaluate.py file.
    ),
    right=accuracy_threshold,  # You can change the threshold here
)

In [23]:
step_cond = ConditionStep(
    name="Check-Accuracy",
    conditions=[cond_lte],
    if_steps=[step_register],
    else_steps=[step_fail]
)

## ✅ Pipeline Definition

In [24]:
pipeline = Pipeline(
    name="TensorFlowKerasPipeline",
    sagemaker_session=sagemaker_session,
    parameters=[processing_instance_type,
                training_instance_type,
                model_approval_status,
                accuracy_threshold
               ],
    steps=[step_process, step_train, step_eval, step_cond ]
)

In [25]:
pipeline.upsert(role_arn=role)

                    INFO     Uploaded None to                                                    ]8;id=724969;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=862322;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#1961\1961]8;;\
                             s3://sagemaker-us-east-1-047922237497/TensorFlowKerasPipeline/code/                   
                             25e96357c3a584da1c420aac79c0a6b11d1704011484e5c3cce7a4ebcc74c0a2/so                   
                             urcedir.tar.gz                                                                        

                    INFO     runproc.sh uploaded to                                              ]8;id=582558;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py\processing.py]8;;\:]8;id=158041;file:///opt/conda/lib/python3.11/site-packages/sagemaker/processing.py#2055\2055]8;;\
                             s3://sagemaker-us-east-1-047922237497/TensorFlowKerasPipeline/code/                   
                             2d81cafc71733a67a70f26bcabb6b9583dfe9d1fd62292923c43f00e784ecf1b/ru                   
                             nproc.sh                                                                              

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=291247;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=212470;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  The input argument instance_type of function                          ]8;id=368577;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=317591;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#422\422]8;;\
                             (sagemaker.image_uris.get_training_image_uri) is a pipeline variable                  
                             (<class 'sagemaker.workflow.parameters.ParameterString'>), which is                   
                             interpreted in pipeline execution time only. As the function needs to                 
                             evaluate the argument value in SDK compile time, the default_value of                 
                             this Parameter object will be used to override it. Please make sure                   
                             the default_value is valid.                                                           

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=152958;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=166810;file:///opt/conda/lib/python3.11/site-packages/sagemaker/image_uris.py#681\681]8;;\
                             instance_type, framework etc.                                                         

...

                    WARNING  Popping out 'ProcessingJobName' from the pipeline definition by       ]8;id=428678;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=48034;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

                    WARNING  No finished training job found associated with this estimator.       ]8;id=286444;file:///opt/conda/lib/python3.11/site-packages/sagemaker/estimator.py\estimator.py]8;;\:]8;id=894691;file:///opt/conda/lib/python3.11/site-packages/sagemaker/estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    WARNING  Popping out 'CertifyForMarketplace' from the pipeline definition since   ]8;id=827789;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/_utils.py\_utils.py]8;;\:]8;id=756039;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/_utils.py#515\515]8;;\
                             it will be overridden in pipeline execution time.                                     

                    WARNING  Popping out 'ModelPackageName' from the pipeline definition by        ]8;id=210126;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py\utilities.py]8;;\:]8;id=187731;file:///opt/conda/lib/python3.11/site-packages/sagemaker/workflow/utilities.py#465\465]8;;\
                             default since it will be overridden at pipeline execution time.                       
                             Please utilize the PipelineDefinitionConfig to persist this field in                  
                             the pipeline definition if desired.                                                   

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:047922237497:pipeline/TensorFlowKerasPipeline',
 'ResponseMetadata': {'RequestId': '6fdb0977-fa45-4c30-bd02-c5a8807bdc56',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6fdb0977-fa45-4c30-bd02-c5a8807bdc56',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '91',
   'date': 'Wed, 19 Feb 2025 23:09:50 GMT'},
  'RetryAttempts': 0}}

In [26]:
execution = pipeline.start()

In [27]:
# execution.wait()
# print("Pipeline execution completed!")

In [28]:
!aws s3 ls s3://aamlops2024/tensorflow-sagemaker-pipeline/artifacts/tensorflow-training-2025-02-19-14-29-58-958/model

                           PRE model/


In [29]:
!aws s3 ls s3://aamlops2024/tensorflow-sagemaker-pipeline/artifacts/pipelines-0qvujrruli3m-TrainTensorFlowModel-IH9POpK40R/output/model.tar.gz

In [30]:
step_train.properties.ModelArtifacts.S3ModelArtifacts

{'_step': <sagemaker.workflow.steps.TrainingStep object at 0x7f0ae43f7e10>, 'step_name': 'TrainTensorFlowModel', 'path': 'ModelArtifacts.S3ModelArtifacts', '_shape_names': ['S3Uri'], '__str__': 'S3Uri'}